In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import plotly.offline as py
import plotly.express as px
from plotly import __version__
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf

from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import GridSearchCV

from sklearn import metrics
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [ ]:
init_notebook_mode(connected=True)
cf.go_offline()

In [ ]:
df = pd.read_csv("loan-data.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
#Checking which columns have more than 50% null values
df_null = pd.DataFrame({'Count': df.isnull().sum(), 'Percent': 100*df.isnull().sum()/len(df)})
df_null[df_null['Percent'] >= 50]

In [ ]:
#Dropping the columns in which 50% of the values are null
df = df.dropna(axis=1, thresh=int(0.50*len(df)))
df.info()

In [ ]:
#Renaming some of the columns
df = df.rename(columns={"loan_amnt": "loan_amount","int_rate": "interest_rate", "annual_inc": "annual_income","funded_amnt":"funded_amount"})

In [ ]:
#Creating a column which is consisting only with the years of the loan issue dates
dt_series = pd.to_datetime(df['issue_d'])
df['year'] = dt_series.dt.year

In [ ]:
df.groupby("year").count()

In [ ]:
#Plot Loans per Year
plt.figure(figsize=(16,8))
sns.barplot('year', 'loan_amount', data=df, palette='tab10')
plt.title('Issuance of Loans Through The Years', fontsize=16)
plt.xlabel('Year', fontsize=14)
plt.ylabel('Average loan amount issued', fontsize=14)

In [ ]:
#Checking the loan statuses
df['loan_status'].unique()

In [ ]:
#Distribution of loan status
m =df['loan_status'].value_counts()
m = m.to_frame()
m.reset_index(inplace=True)
m.columns = ['Loan Status','Count']

fig = px.bar(m, x="Loan Status", y="Count", color="Loan Status")

fig.update_layout(
    autosize=False,
    width=990,
    height=700,
    margin=go.layout.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    legend=go.layout.Legend(
        x=0.7,
        y=1.2,
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        bgcolor="LightSteelBlue",
        bordercolor="Black",
        borderwidth=2
    ),
    title_text='Distribution of Loan Status in our Dataset'
)
fig.show()

In [ ]:
#Distributing the loan status to : bad loans and good loans
bad_loan = ["Charged Off", "Default", "Does not meet the credit policy. Status:Charged Off", "In Grace Period", 
            "Late (16-30 days)", "Late (31-120 days)"]

#Creating function which will change the loan status to bad or good
def loan_condition(status):
    if status in bad_loan:
        return 'Bad Loan'
    else:
        return 'Good Loan'

In [ ]:
#Defining a new column - loan_condition, which will containg values wheter the loan status is good or bad
df['loan_condition'] = np.nan
#Applying the function that we created on the previous block on the new column   
df['loan_condition'] = df['loan_status'].apply(loan_condition)

In [ ]:
df.groupby(['loan_condition',"year"]).count()

In [ ]:
# Plotting the comparison between the Good and the Bad loans
f, ax = plt.subplots(1,2, figsize=(16,8))

colors = ["#3791D7", "#D72626"]
labels ="Good Loans", "Bad Loans"

plt.suptitle('Information on Loan Conditions', fontsize=20)

df["loan_condition"].value_counts().plot.pie(explode=[0,0.25], autopct='%1.2f%%', ax=ax[0], shadow=True, colors=colors, 
                                             labels=labels, fontsize=12, startangle=70)

ax[0].set_title('State of Loan', fontsize=16)
ax[0].set_ylabel('% of Condition of Loans', fontsize=14)

sns.countplot('loan_condition', data=df, ax=ax[1], palette=colors)
ax[1].set_title('Condition of Loans', fontsize=20)
ax[1].set_xticklabels(['Good Loan', 'Bad Loan'], rotation='horizontal')
palette = ["#3791D7", "#E01E1B"]


In [ ]:
#Plotting comparison between the Good and Bad Loans per Year
plt.figure(figsize=(16,8))
sns.countplot(x='year',hue='loan_condition',data=df,palette=palette)

In [ ]:
f, ((ax1, ax2)) = plt.subplots(1, 2)
cmap = plt.cm.coolwarm

by_credit_score = df.groupby(['year', 'grade']).loan_amount.mean()
by_credit_score.unstack().plot(legend=False, ax=ax1, figsize=(14, 4), colormap=cmap)
ax1.set_title('Loans issued by Credit Score', fontsize=14)
    
    
by_inc = df.groupby(['year', 'grade']).interest_rate.mean()
by_inc.unstack().plot(ax=ax2, figsize=(14, 4), colormap=cmap)
ax2.set_title('Interest Rates by Credit Score', fontsize=14)

ax2.legend(bbox_to_anchor=(-1.0, -0.3, 1.7, 0.1), loc=5, prop={'size':12},
           ncol=7, mode="expand", borderaxespad=0.)

In [ ]:
#Relation between interest rate for each type of loan
plt.figure(figsize=(16,6))

ax = sns.lineplot(x="year", y="interest_rate",hue="loan_condition", data=df)
ax = plt.title('Time Series Plot of Interest Rate By Loan Status Through The Years')
ax = plt.xlabel('Year')
ax = plt.ylabel('Interest Rate')

In [ ]:
#Plotting the amount of loans borrowed through the years for each type
import plotly.graph_objs as go
title = 'Amount of Loans borrowed'

labels = bad_loan # All the elements that comprise a bad loan.

len(labels)
colors = ['rgba(236, 112, 99, 1)', 'rgba(235, 152, 78, 1)']

mode_size = [8,8,8,8,8,8]

line_size = [2,2,2,2,2,2]

x_data = [
    sorted(df['year'].unique().tolist())
]

# type of loans
goodloan = df[df['loan_condition'] == 'Good Loan']['loan_amount'].values.tolist()
badloan = df[df['loan_condition'] == 'Bad Loan']['loan_amount'].values.tolist()

y_data = [
    goodloan,
    badloan
]

p_goodloans = go.Scatter(
    x = x_data[0],
    y = y_data[0],
    mode='lines+markers',
    name = 'Good Loan',
    line = dict(
        color = colors[0],
        width = 3)
)

p_badloans = go.Scatter(
    x = x_data[0],
    y = y_data[1],
    mode='lines+markers',
    name = 'Bad Loan',
    line = dict(
        color = colors[1],
        width = 3)
)


data=[p_goodloans, p_badloans]

layout = dict(title = 'Types of Loans <br> (Amount Borrowed Throughout the Years)',
                  xaxis = dict(title = 'Year'),
                  yaxis = dict(title = 'Amount Issued')
              )

fig = dict(data=data, layout=layout)

iplot(fig, filename='line-mode')

In [ ]:
#Making a wordcloud for purpose of the loan,employee title adn home ownership
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

stopwords = set(STOPWORDS)

def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
        background_color='white',
        stopwords=stopwords,
        max_words=200,
        max_font_size=40, 
        scale=3,
        random_state=1 # chosen at random by flipping a coin; it was heads
    ).generate(str(data))

    fig = plt.figure(1, figsize=(12, 12))
    plt.axis('off')
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.show()

show_wordcloud(df[['purpose','emp_title','home_ownership']].dropna())

In [ ]:
#Distributing the interest rate into "High" or "Low" values into a new column- 'interest_payments'
df['interest_payments'] = np.nan
lst = [df]

for col in lst:
    col.loc[col['interest_rate'] <= 13.23, 'interest_payments'] = 'Low'
    col.loc[col['interest_rate'] > 13.23, 'interest_payments'] = 'High'
    

In [ ]:
#Plotting :
    #The impact of interest rate on the condition of the loan
    #The impact of maturity date on interest rates
    #Comparison between the types of interest payments
from scipy.stats import norm

plt.figure(figsize=(20,10))

palette = ['#7e42f5', '#f5b942']
plt.subplot(221)
ax = sns.countplot(x='interest_payments', data=df, 
                  palette=palette, hue='loan_condition')

ax.set_title('The impact of interest rate \n on the condition of the loan', fontsize=14)
ax.set_xlabel('Level of Interest Payments', fontsize=12)
ax.set_ylabel('Count')

plt.subplot(222)
ax1 = sns.countplot(x='interest_payments', data=df, 
                   palette=palette, hue='term')

ax1.set_title('The impact of maturity date \n on interest rates', fontsize=14)
ax1.set_xlabel('Level of Interest Payments', fontsize=12)
ax1.set_ylabel('Count')


plt.subplot(212)
low = df['loan_amount'].loc[df['interest_payments'] == 'Low'].values
high = df['loan_amount'].loc[df['interest_payments'] == 'High'].values


ax2= sns.distplot(low, color='#7e42f5', label='Low Interest Payments', fit=norm, fit_kws={"color":"#483d8b"}) # Dark Blue Norm Color
ax3 = sns.distplot(high, color='#f5b942', label='High Interest Payments', fit=norm, fit_kws={"color":"#c71585"}) #  Red Norm Color
plt.axis([0, 36000, 0, 0.00016])
plt.legend()


plt.show()

In [ ]:
#Plotting the amount of loans issued through the years for each type of bad loan
import plotly.graph_objs as go
title = 'Bad Loans: Loan Statuses'

labels = bad_loan # All the elements that comprise a bad loan.

len(labels)
colors = ['rgba(236, 112, 99, 1)', 'rgba(235, 152, 78, 1)', 'rgba(52, 73, 94, 1)', 'rgba(128, 139, 150, 1)',
         'rgba(255, 87, 51, 1)', 'rgba(255, 195, 0, 1)']

mode_size = [8,8,8,8,8,8]

line_size = [2,2,2,2,2,2]

x_data = [
    sorted(df['year'].unique().tolist()),
    sorted(df['year'].unique().tolist()),
    sorted(df['year'].unique().tolist()),
    sorted(df['year'].unique().tolist()), 
    sorted(df['year'].unique().tolist()),
    sorted(df['year'].unique().tolist()),
]

# type of loans
charged_off = df['loan_amount'].loc[df['loan_status'] == 'Charged Off'].values.tolist()
defaults = df['loan_amount'].loc[df['loan_status'] == 'Default'].values.tolist()
not_credit_policy = df['loan_amount'].loc[df['loan_status'] == 'Does not meet the credit policy. Status:Charged Off'].values.tolist()
grace_period = df['loan_amount'].loc[df['loan_status'] == 'In Grace Period'].values.tolist()
short_late = df['loan_amount'].loc[df['loan_status'] == 'Late (16-30 days)'].values.tolist()
long_late = df['loan_amount'].loc[df['loan_status'] == 'Late (31-120 days)'].values.tolist()

y_data = [
    charged_off,
    defaults,
    not_credit_policy,
    grace_period,
    short_late,
    long_late,
]

p_charged_off = go.Scatter(
    x = x_data[0],
    y = y_data[0],
    name = 'A. Charged Off',
    line = dict(
        color = colors[0],
        width = 3,
        dash='dash')
)

p_defaults = go.Scatter(
    x = x_data[1],
    y = y_data[1],
    name = 'A. Defaults',
    line = dict(
        color = colors[1],
        width = 3,
        dash='dash')
)

p_credit_policy = go.Scatter(
    x = x_data[2],
    y = y_data[2],
    name = 'Not Meet C.P',
    line = dict(
        color = colors[2],
        width = 3,
        dash='dash')
)

p_graced = go.Scatter(
    x = x_data[3],
    y = y_data[3],
    name = 'A. Graced Period',
    line = dict(
        color = colors[3],
        width = 3,
        dash='dash')
)

p_short_late = go.Scatter(
    x = x_data[4],
    y = y_data[4],
    name = 'Late (16-30 days)',
    line = dict(
        color = colors[4],
        width = 3,
        dash='dash')
)

p_long_late = go.Scatter(
    x = x_data[5],
    y = y_data[5],
    name = 'Late (31-120 days)',
    line = dict(
        color = colors[5],
        width = 3,
        dash='dash')
)




data=[p_charged_off, p_defaults, p_credit_policy, p_graced, p_short_late, p_long_late]

layout = dict(title = 'Types of Bad Loans <br> (Amount Borrowed Throughout the Years)',
              xaxis = dict(title = 'Year'),
              yaxis = dict(title = 'Amount Issued'),
              )

fig = dict(data=data, layout=layout)

iplot(fig, filename='line-mode')

In [ ]:
#Plotting the loans issued per different type of home ownership
import plotly.express as px
fig = px.box(df, x="year", y="loan_amount", color="home_ownership", notched=True)

fig.update_layout(
    autosize=False,
    width=1500,
    height=700,
    margin=go.layout.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    legend=go.layout.Legend(
        x=0,
        y=1,
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        bgcolor="LightSteelBlue",
        bordercolor="Black",
        borderwidth=2
    )
)
fig.show()

In [ ]:
#Distributing to good and bad loans for each purpose they have been issued
df['purpose'].value_counts()

# Education, renewable energy, wedding are the purposed that contains highest bad loans percent wise.

purpose_condition = round(pd.crosstab(df['loan_condition'], df['purpose']).apply(lambda x: x/x.sum() * 100), 2)

purpose_bad_loans = purpose_condition.values[0].tolist()
purpose_good_loans = purpose_condition.values[1].tolist()
purpose = purpose_condition.columns


bad_plot = go.Bar(
    x=purpose,
    y=purpose_bad_loans,
    name = 'Bad Loans',
    text='%',
    marker=dict(
        color='rgba(213, 250, 5, 0.7)',
        line = dict(
            color='rgba(213, 250, 5, 1.0)',
            width=2
        )
    )
)

good_plot = go.Bar(
    x=purpose,
    y=purpose_good_loans,
    name='Good Loans',
    text='%',
    marker=dict(
        color='rgba(187, 4, 212, 0.7)',
        line = dict(
            color='rgba(187, 4, 212, 1.0)',
            width=2
        )
    )
)


data = [bad_plot, good_plot]

layout = go.Layout(
    title='Condition of Loan by Purpose',
    xaxis=dict(
        title=''
    ),
    yaxis=dict(
        title='% of the Loan',
    ),
    paper_bgcolor='#c8cfcf',
    plot_bgcolor='#c8cfcf',
    showlegend=True
)

fig = dict(data=data, layout=layout)
iplot(fig, filename='condition_purposes')

In [ ]:
#Creating a new column - 'emp-length_int', which represents the lenght of the employee in numbers
employment_length = ['10+ years', '< 1 year', '1 year', '3 years', '8 years', '9 years',
                    '4 years', '5 years', '6 years', '2 years', '7 years', 'n/a']

# Create a new column and convert emp_length to integers.

lst = [df]
df['emp_length_int'] = np.nan

for col in lst:
    col.loc[col['emp_length'] == '10+ years', "emp_length_int"] = 10
    col.loc[col['emp_length'] == '9 years', "emp_length_int"] = 9
    col.loc[col['emp_length'] == '8 years', "emp_length_int"] = 8
    col.loc[col['emp_length'] == '7 years', "emp_length_int"] = 7
    col.loc[col['emp_length'] == '6 years', "emp_length_int"] = 6
    col.loc[col['emp_length'] == '5 years', "emp_length_int"] = 5
    col.loc[col['emp_length'] == '4 years', "emp_length_int"] = 4
    col.loc[col['emp_length'] == '3 years', "emp_length_int"] = 3
    col.loc[col['emp_length'] == '2 years', "emp_length_int"] = 2
    col.loc[col['emp_length'] == '1 year', "emp_length_int"] = 1
    col.loc[col['emp_length'] == '< 1 year', "emp_length_int"] = 0.5
    col.loc[col['emp_length'] == 'n/a', "emp_length_int"] = 0

In [ ]:
#Defining new column - 'income_category', which relates to the type('Low','Medium' or 'High')
df['income_category'] = np.nan
lst = [df]

for col in lst:
    col.loc[col['annual_income'] <= 100000, 'income_category'] = 'Low'
    col.loc[(col['annual_income'] > 100000) & (col['annual_income'] <= 200000), 'income_category'] = 'Medium'
    col.loc[col['annual_income'] > 200000, 'income_category'] = 'High'

In [ ]:
group_income_purpose = df.groupby(['income_category', 'purpose'], as_index=False).interest_rate.mean()
group_dti_purpose = df.groupby(['income_category', 'purpose'], as_index=False).loan_amount.mean()
loan_a = group_dti_purpose['loan_amount'].values



# High Car 10.32 15669
new_groupby = group_income_purpose.assign(total_loan_amount=loan_a)
sort_group_income_purpose = new_groupby.sort_values(by="income_category", ascending=True)

In [ ]:
loan_count = df.groupby(['income_category', 'purpose'])['loan_condition'].apply(lambda x: x.value_counts())
d={"loan_c": loan_count}
loan_c_df = pd.DataFrame(data=d).reset_index()
loan_c_df = loan_c_df.rename(columns={"level_2": "loan_condition"})


# Good loans & Bad Loans
good_loans = loan_c_df.loc[loan_c_df['loan_condition'] == "Good Loan"].sort_values(by="income_category", ascending=True)
bad_loans = loan_c_df.loc[loan_c_df['loan_condition'] == "Bad Loan"].sort_values(by="income_category", ascending=True)
sort_group_income_purpose['good_loans_count'] = good_loans['loan_c'].values
sort_group_income_purpose['bad_loans_count'] = bad_loans['loan_c'].values
sort_group_income_purpose['total_loans_issued'] = (good_loans['loan_c'].values + bad_loans['loan_c'].values)
sort_group_income_purpose['bad/good ratio (%)'] = np.around(bad_loans['loan_c'].values / (bad_loans['loan_c'].values + good_loans['loan_c'].values), 4) * 100
final_df = sort_group_income_purpose.sort_values(by='income_category', ascending=True)
final_df.style.background_gradient('Greens')

In [ ]:
#Creating a column - 'loan_condition_int',which can be '1' - related to a 'Good Loan' and '0' for a 'Bad Loan'
lst = [df]
df['loan_condition_int'] = np.nan

for col in lst:
    col.loc[df['loan_condition'] == 'Good Loan', 'loan_condition_int'] = 0 # Negative (Bad Loan)
    col.loc[df['loan_condition'] == 'Bad Loan', 'loan_condition_int'] = 1 # Positive (Good Loan)
    
# Convert from float to int the column (This is our label)  
df['loan_condition_int'] = df['loan_condition_int'].astype(int)

In [ ]:
#Diisplaying :
    #Loan Issued Per Inc. Category
    #Good/Bad Loan Per Inc. Category
    #Employee Length Per Inc. Category
    #Interest Rate Per Inc. Category
    
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=2, start_cell="bottom-left",
                    subplot_titles=("Loan Issued Per Inc. Category","Good/Bad Loan Per Inc. Category",
                                    "Employee Length Per Inc. Category","Interest Rate Per Inc. Category"))

fig.add_trace(go.Violin(
            x=df["income_category"],
            y=df["loan_amount"]),
              row=1, col=1)

fig.add_trace(go.Violin(
            x=df["income_category"], 
            y=df["loan_condition"]),
              row=1, col=2)

fig.add_trace(go.Violin(
            x=df["income_category"],
            y=df["emp_length_int"]),
              row=2, col=1)

fig.add_trace(go.Violin(
            x=df["income_category"], 
            y=df["interest_rate"]),
              row=2, col=2)

fig.update_layout(showlegend=False, title_text="Income Distribution")

fig.show()

In [ ]:
title_mask = df.emp_title.value_counts()[:20].index.values 
cm = sns.light_palette("green", as_cmap=True)

dtfig = round(pd.crosstab(df[df['emp_title'].isin(title_mask)]['emp_title'], 
                  df[df['emp_title'].isin(title_mask)]['sub_grade'], 
                  normalize='index') * 100,2)
dtfig.style.background_gradient(cmap = cm)

In [ ]:
#Plotting top 40 employment titles that had loans
trace0 = go.Bar(
    x = df.emp_title.value_counts()[:40].index.values,
    y = df.emp_title.value_counts()[:40].values,
    marker=dict(
        color=df.emp_title.value_counts()[:40].values
    ),
)

data = [trace0]

layout = go.Layout(
    yaxis=dict(
        title='Count'
    ),
    xaxis=dict(
        title='Employment name'
    ),
    title='TOP 40 Employment Title'
)

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='emp-title-bar')

In [ ]:
df['addr_state'].unique()

In [ ]:
# Make a list with each of the regions by state and out them into a new column - 'region'.

west = ['CA', 'OR', 'UT','WA', 'CO', 'NV', 'AK', 'MT', 'HI', 'WY', 'ID']
south_west = ['AZ', 'TX', 'NM', 'OK']
south_east = ['GA', 'NC', 'VA', 'FL', 'KY', 'SC', 'LA', 'AL', 'WV', 'DC', 'AR', 'DE', 'MS', 'TN' ]
mid_west = ['IL', 'MO', 'MN', 'OH', 'WI', 'KS', 'MI', 'SD', 'IA', 'NE', 'IN', 'ND']
north_east = ['CT', 'NY', 'PA', 'NJ', 'RI','MA', 'MD', 'VT', 'NH', 'ME']



df['region'] = np.nan

def finding_regions(state):
    if state in west:
        return 'West'
    elif state in south_west:
        return 'SouthWest'
    elif state in south_east:
        return 'SouthEast'
    elif state in mid_west:
        return 'MidWest'
    elif state in north_east:
        return 'NorthEast'
    


df['region'] = df['addr_state'].apply(finding_regions)

In [ ]:
df['complete_date'] = pd.to_datetime(df['issue_d'])

group_dates = df.groupby(['complete_date', 'region'], as_index=False).sum()

group_dates['issue_d'] = [month.to_period('M') for 
                          month in group_dates['complete_date']]

group_dates = group_dates.groupby(['issue_d', 'region'], as_index=False).sum()
group_dates = group_dates.groupby(['issue_d', 'region'], as_index=False).sum()
group_dates['loan_amount'] = group_dates['loan_amount']/1000


df_dates = pd.DataFrame(data=group_dates[['issue_d','region','loan_amount']])

In [ ]:
#Plotting the:
    #Average Interest Rate by Region
    #Average Employment Length by Region
    #Average Debt-to-Income by Region
    #Average Annual Income by Region

sns.set_style('whitegrid')

f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)
cmap = 'cividis'

by_interest_rate = df.groupby(['year', 'region']).interest_rate.mean()
by_interest_rate.unstack().plot(kind='area', stacked=True, colormap=cmap, grid=False, legend=False, ax=ax1, figsize=(16,12))
ax1.set_title('Average Interest Rate by Region', fontsize=14)


by_employment_length = df.groupby(['year', 'region']).emp_length_int.mean()
by_employment_length.unstack().plot(kind='area', stacked=True, colormap=cmap, grid=False, legend=False, ax=ax2, figsize=(16,12))
ax2.set_title('Average Employment Length by Region', fontsize=14)
# plt.xlabel('Year of Issuance', fontsize=14)

by_dti = df.groupby(['year', 'region']).dti.mean()
by_dti.unstack().plot(kind='area', stacked=True, colormap=cmap, grid=False, legend=False, ax=ax3, figsize=(16,12))
ax3.set_title('Average Debt-to-Income by Region', fontsize=14)

by_income = df.groupby(['year', 'region']).annual_income.mean()
by_income.unstack().plot(kind='area', stacked=True, colormap=cmap, grid=False, ax=ax4, figsize=(16,12))
ax4.set_title('Average Annual Income by Region', fontsize=14)
ax4.legend(bbox_to_anchor=(-1.0, -0.5, 1.8, 0.1), loc=10,prop={'size':12},
           ncol=5, mode="expand", borderaxespad=0.)

In [ ]:
# Grouping by our metrics
by_loan_amount = df.groupby(['region','addr_state'], as_index=False).loan_amount.sum()
by_interest_rate = df.groupby(['region', 'addr_state'], as_index=False).interest_rate.mean()
by_income = df.groupby(['region', 'addr_state'], as_index=False).annual_income.mean()



# Take the values to a list for visualization purposes.
states = by_loan_amount['addr_state'].values.tolist()
average_loan_amounts = by_loan_amount['loan_amount'].values.tolist()
average_interest_rates = by_interest_rate['interest_rate'].values.tolist()
average_annual_income = by_income['annual_income'].values.tolist()

from collections import OrderedDict

metrics_data = OrderedDict([('state_codes', states),
                            ('issued_loans', average_loan_amounts),
                            ('interest_rate', average_interest_rates),
                            ('annual_income', average_annual_income)])
                     

metrics_df = pd.DataFrame.from_dict(metrics_data)
metrics_df = metrics_df.round(decimals=2)
metrics_df.head()

In [ ]:
#Plotting Loans Issued Per States
import plotly.graph_objs as go


for col in metrics_df.columns:
    metrics_df[col] = metrics_df[col].astype(str)
    
scl = [[0.0, 'rgb(210, 241, 198)'],[0.2, 'rgb(188, 236, 169)'],[0.4, 'rgb(171, 235, 145)'],\
            [0.6, 'rgb(140, 227, 105)'],[0.8, 'rgb(105, 201, 67)'],[1.0, 'rgb(59, 159, 19)']]

metrics_df['text'] = metrics_df['state_codes'] + '<br>' +\
'Average loan interest rate: ' + metrics_df['interest_rate'] + '<br>'+\
'Average annual income: ' + metrics_df['annual_income'] 


data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = metrics_df['state_codes'],
        z = metrics_df['issued_loans'], 
        locationmode = 'USA-states',
        text = metrics_df['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "$s USD")
        ) ]


layout = dict(
    title = 'Lending Clubs Issued Loans <br> (A Perspective for the Business Operations)',
    geo = dict(
        scope = 'usa',
        projection=dict(type='albers usa'),
        showlakes = True,
        lakecolor = 'rgb(255, 255, 255)')
)

fig = dict(data=data, layout=layout)
iplot(fig, filename='d3-cloropleth-map')

In [ ]:
badLoansPerState = df[df['loan_condition']=='Bad Loan'].groupby('addr_state')['loan_condition'].count().sort_values()

In [ ]:
#Plotting Bad Loans Issued Per State
fig = px.bar(df, x=badLoansPerState.values, y=badLoansPerState.keys(), height=600,
            color=badLoansPerState.keys(),
            labels={'y':'State','x':'Number of loans'},
            title='Number of bad loans per state',
            orientation='h')
fig.show()

In [ ]:
goodLoansPerState = df[df['loan_condition']=='Good Loan'].groupby('addr_state')['loan_condition'].count().sort_values()

In [ ]:
#Plotting Good Loans Issued Per State
fig = px.bar(df, x=goodLoansPerState.values, y=goodLoansPerState.keys(), height=600,
            color=goodLoansPerState.keys(),
            labels={'y':'State','x':'Number of loans'},
            title='Number of good loans per state',
            orientation='h')
fig.show()

In [ ]:
#Preparing the data by filtering the values that might be usefult for the prediction
dfPred = df.filter(['loan_amount','term','interest_rate','installment','grade','sub_grade','emp_length','home_ownership',
                    'annual_income','purpose','funded_amount','tot_cur_bal','dti','delinq_2yrs','loan_status',
                    'recoveries','collection_recovery_fee','total_rec_prncp','out_prncp','last_pymnt_d'])

In [ ]:
dfCleared = dfPred.dropna()

In [ ]:
sns.heatmap(dfCleared.isnull())

In [ ]:
dfCleared.info()

In [ ]:
dfCleared['loan_status'] = dfCleared['loan_status'].apply(loan_condition)

In [ ]:
dfCleared

In [ ]:
dfCleared['interest_rate'] = dfCleared['interest_rate'].map(lambda x: x/100)

In [ ]:
#Plotting the correllation between the data columns 
plt.figure(figsize=(20,20))
sns.set_context("paper", font_scale=1)
##finding the correllation matrix and changing the categorical data to category for the plot.
sns.heatmap(dfCleared.assign(
                        loan_status=dfCleared.loan_status.astype('category').cat.codes.astype(float),
                        grade=dfCleared.grade.astype('category').cat.codes.astype(float),
                        sub_grade=dfCleared.sub_grade.astype('category').cat.codes.astype(float),
                        term=dfCleared.term.astype('category').cat.codes.astype(float),
                        emp_length=dfCleared.emp_length.astype('category').cat.codes.astype(float),
                        home_ownership=dfCleared.home_ownership.astype('category').cat.codes.astype(float),
                        purpose=dfCleared.purpose.astype('category').cat.codes.astype(float),
                        delinq_2yrs=dfCleared.delinq_2yrs.astype('category').cat.codes.astype(float),
                        last_pymnt_d=dfCleared.last_pymnt_d.astype('category').cat.codes.astype(float),
                        funded_amount=dfCleared.funded_amount.astype(float)).corr(),
                        annot=True, cmap='bwr',vmin=-1, vmax=1, square=True, linewidths=0.5)

In [ ]:
#Dropping the columns which will not be useful
dfCleared.drop(['installment','grade','sub_grade','term','funded_amount','collection_recovery_fee']
           , axis=1, inplace = True)

In [ ]:
dfCleared

In [ ]:
sns.pairplot(dfCleared,hue='loan_status')

In [ ]:
#Preparing the data by categorizing the columns with type of values object
dfCleared["loan_status"] = dfCleared["loan_status"].replace(['Good Loan', 'Bad Loan'], [1,0])
dfCleared.purpose=dfCleared.purpose.astype('category').cat.codes
dfCleared.delinq_2yrs=dfCleared.delinq_2yrs.astype('category').cat.codes
dfCleared.home_ownership=dfCleared.home_ownership.astype('category').cat.codes
dfCleared.emp_length=dfCleared.emp_length.astype('category').cat.codes
dfCleared.last_pymnt_d=dfCleared.last_pymnt_d.astype('category').cat.codes

final_data = dfCleared

In [ ]:
final_data

In [ ]:
#Splitting the data into train an test set
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split

X = final_data.drop('loan_status',axis=1)
y = final_data['loan_status']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35,random_state=101)


In [ ]:
#Crate a coss validation function for evaluation of the model
def evaluate_cross_validation(clf,X,y,K):
    kf = KFold(K, shuffle=True, random_state=0)
    
    scores = cross_val_score(clf, X, y, cv=kf)
    
    print("Scores: \n"  , scores)
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
def confusion_matrix_display(confusion_score):
    f, ax_m = plt.subplots(1, 3, figsize=(18, 6))

    cm_pred_label_sum = confusion_score.sum(axis=0)
    cm_true_label_sum = confusion_score.sum(axis=1)

    model_precision, model_recall = np.empty([2, 2], dtype=float), np.empty([2, 2], dtype=float)
    model_precision[0][0], model_precision[1][0] = confusion_score[0][0] / cm_pred_label_sum[0], \
                                                         confusion_score[1][0] / cm_pred_label_sum[0] 
    model_precision[0][1], model_precision[1][1] = confusion_score[0][1] / cm_pred_label_sum[1], \
                                                         confusion_score[1][1] / cm_pred_label_sum[1]
    model_recall[0][0], model_recall[0][1] = confusion_score[0][0] / cm_true_label_sum[0], \
                                                   confusion_score[0][1] / cm_true_label_sum[0]
    model_recall[1][0], model_recall[1][1] = confusion_score[1][0] / cm_true_label_sum[1], \
                                                   confusion_score[1][1] / cm_true_label_sum[1]
    sns.heatmap(confusion_score, annot=True, fmt="d", cmap="Blues", ax=ax_m[0], square=True, linewidths=0.5)
    sns.heatmap(model_precision, annot=True, fmt=".5f", cmap="Blues", ax=ax_m[1], square=True, linewidths=0.5)
    sns.heatmap(model_recall, annot=True, fmt=".5f", cmap="Blues", ax=ax_m[2], square=True, linewidths=0.5)
    ax_m[0].set_title("confusion matrix", fontsize=16)
    ax_m[1].set_title("model precision", fontsize=16)
    ax_m[2].set_title("model recall", fontsize=16)
    ax_m[0].set_xlabel("Predicted label", fontsize=16)
    ax_m[0].set_ylabel("True label", fontsize=16)
    ax_m[1].set_xlabel("Predicted label", fontsize=16)
    ax_m[1].set_ylabel("True label", fontsize=16)
    ax_m[2].set_xlabel("Predicted label", fontsize=16)
    ax_m[2].set_ylabel("True label", fontsize=16)
    plt.show()
    plt.close()

In [ ]:
#Decision Tree CLassifier Model Fitting
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier()
# dtree.fit(X_train,y_train)
predictions = dtree.predict(X_test)
dtree_confusion_score = confusion_matrix(y_test, predictions)
print(accuracy_score(y_test, predictions))
confusion_matrix_display(dtree_confusion_score)

In [ ]:
#Random Forest Classifier Fitting
from sklearn.ensemble import RandomForestClassifier
rfc2 = RandomForestClassifier(n_estimators=100)
rfc2.fit(X_train, y_train)
rfc_pred = rfc2.predict(X_test)

rf_test_acc = accuracy_score(y_test, rfc_pred)
rf_classification_score = classification_report(y_test, rfc_pred)
print("Rf model test accuracy:{:.4f}".format(rf_test_acc))
print("rf model classification_score:\n", rf_classification_score)
rf_confusion_score = confusion_matrix(y_test, rfc_pred)

confusion_matrix_display(rf_confusion_score)

#Grid Search for the best number of estimators 

# params = {'n_estimators': [50, 100, 200, 400, 600, 800]}
# grid_search = GridSearchCV(RandomForestClassifier(), param_grid=params,
#                                    scoring='accuracy', cv=5, n_jobs=-1)
# grid_search.fit(X_train, y_train)
# print(grid_search.best_params_)
# print(grid_search.best_score_)

In [ ]:
#Applying Neural Network algorithm with tensorflow
import tensorflow as tf

In [ ]:
dfCleared.columns

In [ ]:
#Feature selection
loan_amount = tf.feature_column.numeric_column("loan_amount")
interest_rate = tf.feature_column.numeric_column('interest_rate')
emp_length = tf.feature_column.numeric_column('emp_length')
home_ownership =tf.feature_column.numeric_column('home_ownership')
annual_income = tf.feature_column.numeric_column("annual_income")
purpose = tf.feature_column.numeric_column('purpose')
tot_cur_bal = tf.feature_column.numeric_column('tot_cur_bal')
dti =tf.feature_column.numeric_column('dti')
delinq_2yrs = tf.feature_column.numeric_column('delinq_2yrs')
recoveries = tf.feature_column.numeric_column('recoveries')
total_rec_prncp = tf.feature_column.numeric_column('total_rec_prncp')
out_prncp = tf.feature_column.numeric_column('out_prncp')
last_pymnt_d = tf.feature_column.numeric_column('last_pymnt_d')

In [ ]:
feat_cols = [loan_amount,interest_rate,emp_length,home_ownership,annual_income,purpose,tot_cur_bal,dti,delinq_2yrs,recoveries,
             total_rec_prncp,out_prncp,last_pymnt_d]

In [ ]:
classifier = tf.estimator.DNNClassifier(hidden_units=[1500], n_classes=2,feature_columns=feat_cols)

In [ ]:
input_func = tf.compat.v1.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=20,shuffle=True)

In [ ]:
classifier.train(input_fn=input_func,steps=10000)

In [ ]:
pred_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),shuffle=False)

In [ ]:
note_predictions = list(classifier.predict(input_fn=pred_fn))

In [ ]:
note_predictions[0]

In [ ]:
final_preds  = []
for pred in note_predictions:
    final_preds.append(pred['class_ids'][0])

In [ ]:
nn_confusion_score = confusion_matrix(y_test, final_preds)
print(nn_confusion_score)

In [ ]:
print(classification_report(y_test,final_preds))
print(accuracy_score(y_test, final_preds))

confusion_matrix_display(nn_confusion_score)

In [ ]:
%load_ext tensorboard

In [ ]:
from datetime import datetime
from tensorflow import keras

#Defining a keras model
model = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(1500, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

# Define the Keras TensorBoard callback.
logdir="logs\\fit\\" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

#Converting to numpy array
X_train=np.asarray(X_train) 
y_train= np.asarray(y_train) 

# Train the model.
model.fit(
    X_train,
    y_train, 
    batch_size=20,
    epochs=4, 
    callbacks=[tensorboard_callback])


In [ ]:
import os
logs_base_dir = "./logs"
os.makedirs(logs_base_dir, exist_ok=True)
%tensorboard --logdir {logs_base_dir}  --host localhost

In [ ]:
# !TASKKILL -F /PID 15264